In [1]:
from rekall import IntervalSetMapping, IntervalSet, Interval
from rekall.bounds import Bounds1D
from rekall.predicates import *

In [2]:
import numpy as np

In [60]:
a = np.load('/dfs/scratch1/danfu/rekall_experiments/heartmri/L_dev.npy')

In [61]:
b = np.load('/dfs/scratch1/danfu/rekall_experiments/heartmri/y_dev.npy')

In [62]:
a.T[6:12]

array([[ 0.,  0.,  0.,  1.,  1.],
       [ 1.,  1.,  0.,  1.,  1.],
       [ 1.,  1.,  1.,  0.,  1.],
       [ 1.,  1.,  1.,  0.,  1.],
       [ 1.,  1.,  1.,  0.,  0.],
       [ 1., -1.,  1.,  0., -1.]])

In [63]:
b.shape[0]

636

In [64]:
a.T.shape

(636, 5)

In [65]:
gt_ism_frame = IntervalSetMapping({
    i: IntervalSet([
        Interval(Bounds1D(frame, frame+1), payload = b.tolist()[i*6 + frame])
        for frame in range(6)
    ])
    for i in range(int(b.shape[0] / 6))
})

In [66]:
gt_ism_patient = gt_ism_frame.coalesce(
    ('t1', 't2'), Bounds1D.span
)

In [67]:
lf_ism = IntervalSetMapping({
    i: IntervalSet([
        Interval(Bounds1D(frame, frame+1), payload=lfs)
        for frame, lfs in enumerate(a.T.tolist()[i * 6:i*6 + 6])
    ])
    for i in range(int(a.T.shape[0] / 6))
})

In [68]:
frame_mv = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [1 if intrvl['payload'].count(1) > intrvl['payload'].count(-1) else -1]
    )
)

In [12]:
patient_mv = frame_mv.coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = 1 if intrvl['payload'].count(1) > intrvl['payload'].count(-1) else -1
    )
)

In [13]:
len(patient_mv.filter(payload_satisfies(lambda p: p == 1)))

40

In [14]:
def tp_fp_fn_patient(predicted, gt):
    tp = predicted.filter(
        payload_satisfies(lambda p: p == 1)
    ).filter_against(
        gt,
        predicate = lambda i1, i2: i1['payload'] == i2['payload']
    )
    
    fp = predicted.filter(
        payload_satisfies(lambda p: p == 1)
    ).minus(tp)
    
    fn = gt.filter(
        payload_satisfies(lambda p: p == 1)
    ).minus(
        predicted.filter(
            payload_satisfies(lambda p: p == 1)
        )
    )
    
    return tp, fp, fn

In [15]:
def prf1_patient(predicted, gt):
    tp = predicted.filter(
        payload_satisfies(lambda p: p == 1)
    ).filter_against(
        gt,
        predicate = lambda i1, i2: i1['payload'] == i2['payload']
    )
    
    fp = predicted.filter(
        payload_satisfies(lambda p: p == 1)
    ).minus(tp)
    
    fn = gt.filter(
        payload_satisfies(lambda p: p == 1)
    ).minus(
        predicted.filter(
            payload_satisfies(lambda p: p == 1)
        )
    )
    
    pre = len(tp) / (len(tp) + len(fp))
    rec = len(tp) / (len(tp) + len(fn))
    f1 = 2 * pre * rec / (pre + rec)
    
    return pre, rec, f1, len(tp), len(fp), len(fn)

In [16]:
prf1_patient(patient_mv, gt_ism_patient)

(0.15, 1.0, 0.2608695652173913, 6, 34, 0)

In [17]:
threshold_frame = 0
threshold_seq = 0

In [18]:
attempt2 = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [
            1 if intrvl['payload'].count(1) > intrvl['payload'].count(-1) + threshold_frame
            else -1]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            1 if intrvl['payload'].count(1) > intrvl['payload'].count(-1) + threshold_seq
            else -1
        )
    )
)

In [19]:
prf1_patient(attempt2, gt_ism_patient)

(0.15, 1.0, 0.2608695652173913, 6, 34, 0)

In [72]:
second_lf_all_1 = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][1]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            1 if intrvl['payload'].count(1) == 6
            else -1
        )
    )
)

In [73]:
prf1_patient(second_lf_all_1, gt_ism_patient)

(0.36363636363636365, 0.6666666666666666, 0.4705882352941177, 4, 7, 2)

In [76]:
tp, fp, fn = tp_fp_fn_patient(second_lf_all_1, gt_ism_patient)

In [77]:
for k in tp:
    print_lfs(lf_ism[k])

[[1. 1. 1. 0. 0.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]]
[[ 0.  1.  0.  1.  0.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  1. -1.]]
[[1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
[[0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]]


In [80]:
for k in fn:
    print_lfs(lf_ism[k])

[[ 0.  0.  0.  1.  1.]
 [ 1.  1.  0.  1.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  0.]
 [ 1. -1.  1.  0. -1.]]
[[-1.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]


In [78]:
for k in fp:
    print_lfs(lf_ism[k])

[[0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]]
[[ 0.  1.  0.  1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0. -1.  0.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0. -1.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  1.  0. -1.]]
[[-1.  1.  0.  0.  0.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0. -1. -1.]]
[[-1.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0. -1.  1.]]


In [81]:
second_five_fifth_all = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][1]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            1 if intrvl['payload'].count(1) == 5
            else -1
        )
    )
).join(
    lf_ism.map(
        lambda intrvl: Interval(
            intrvl['bounds'],
            payload = [intrvl['payload'][4]]
        )
    ).coalesce(
        ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
    ).map(
        lambda intrvl: Interval(
            intrvl['bounds'],
            payload = (
                1 if intrvl['payload'].count(1) == 6
                else -1
            )
        )
    ),
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 and i2['payload'] == 1 else -1
    )
)

In [84]:
prf1_patient(second_five_fifth_all, gt_ism_patient)

(0.25, 0.16666666666666666, 0.2, 1, 3, 5)

In [112]:
three_with_four_positive = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [1 if intrvl['payload'].count(1) >= 4 else -1]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            1 if intrvl['payload'].count(1) >= 3
            else -1
        )
    )
)

In [113]:
prf1_patient(three_with_four_positive, gt_ism_patient)

(0.75, 0.5, 0.6, 3, 1, 3)

In [114]:
tp, fp, fn = tp_fp_fn_patient(three_with_four_positive, gt_ism_patient)

In [115]:
for k in tp:
    print_lfs(lf_ism[k])

[[1. 1. 1. 0. 0.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]]
[[ 0.  0.  0.  1.  1.]
 [ 1.  1.  0.  1.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  0.]
 [ 1. -1.  1.  0. -1.]]
[[1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [116]:
for k in fp:
    print_lfs(lf_ism[k])

[[ 1. -1.  1.  1. -1.]
 [ 1. -1.  1.  0.  1.]
 [ 1.  1.  1.  0. -1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]]


In [117]:
two_columns_multiple_negative = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [1 if lf == -1 else 0 for lf in intrvl['payload']]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: [i + j for i, j in zip(p1, p2)]
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            -1 if len([1 for count in intrvl['payload'] if count > 1]) > 1
            else 1
        )
    )
)

In [118]:
three_four_positive_columns_not_negative = three_with_four_positive.join(
    two_columns_multiple_negative,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 and i2['payload'] == 1 else -1
    )
)

In [119]:
prf1_patient(three_four_positive_columns_not_negative, gt_ism_patient)

(1.0, 0.5, 0.6666666666666666, 3, 0, 3)

In [120]:
tp, fp, fn = tp_fp_fn_patient(three_four_positive_columns_not_negative, gt_ism_patient)

In [123]:
for k in tp:
    print_lfs(lf_ism[k])

[[1. 1. 1. 0. 0.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]]
[[ 0.  0.  0.  1.  1.]
 [ 1.  1.  0.  1.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  0.]
 [ 1. -1.  1.  0. -1.]]
[[1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [121]:
for k in fn:
    print_lfs(lf_ism[k])

[[-1.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0.  1.  0.  1.  0.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  1. -1.]]
[[0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]]


In [125]:
second_all_fifth_all = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][1]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            1 if intrvl['payload'].count(1) == 6
            else -1
        )
    )
).join(
    lf_ism.map(
        lambda intrvl: Interval(
            intrvl['bounds'],
            payload = [intrvl['payload'][4]]
        )
    ).coalesce(
        ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
    ).map(
        lambda intrvl: Interval(
            intrvl['bounds'],
            payload = (
                1 if intrvl['payload'].count(1) == 6
                else -1
            )
        )
    ),
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 and i2['payload'] == 1 else -1
    )
)

In [126]:
prf1_patient(second_all_fifth_all, gt_ism_patient)

(0.4, 0.3333333333333333, 0.3636363636363636, 2, 3, 4)

In [130]:
attempt_final = three_four_positive_columns_not_negative.join(
    second_all_fifth_all,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 or i2['payload'] == 1 else -1
    )
)

In [131]:
prf1_patient(attempt_final, gt_ism_patient)

(0.5714285714285714, 0.6666666666666666, 0.6153846153846153, 4, 3, 2)

In [132]:
tp, fp, fn = tp_fp_fn_patient(attempt_final, gt_ism_patient)

In [133]:
for k in fp:
    print_lfs(lf_ism[k])

[[0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]]
[[ 0.  1.  0.  1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0. -1.  1.]]


In [134]:
for k in tp:
    print_lfs(lf_ism[k])

[[1. 1. 1. 0. 0.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]]
[[ 0.  0.  0.  1.  1.]
 [ 1.  1.  0.  1.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  0.]
 [ 1. -1.  1.  0. -1.]]
[[1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
[[0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]]


In [153]:
first_nonzero = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][0]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            1 if intrvl['payload'].count(0) < 4
            else -1
        )
    )
)

fourth_positive = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][3]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            1 if sum(intrvl['payload']) > 0
            else -1
        )
    )
)

first_nonzero_or_fourth_positive = first_nonzero.join(
    fourth_positive,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 or i2['payload'] == 1 else -1
    )
)

second_all_fifth_all_stricter = second_all_fifth_all.join(
    first_nonzero_or_fourth_positive,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 and i2['payload'] == 1 else -1
    )
)

attempt_really_final = three_four_positive_columns_not_negative.join(
    second_all_fifth_all_stricter,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 or i2['payload'] == 1 else -1
    )
)

In [154]:
prf1_patient(attempt_really_final, gt_ism_patient)

(1.0, 0.5, 0.6666666666666666, 3, 0, 3)

In [20]:
second_lf_unsure = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][1]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            1 if intrvl['payload'].count(1) > 4
            else -1
        )
    )
)

In [21]:
prf1_patient(second_lf_unsure, gt_ism_patient)

(0.14705882352941177, 0.8333333333333334, 0.25, 5, 29, 1)

In [22]:
fourth_lf_negative = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][3]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            -1 if intrvl['payload'].count(-1) in (2, 3)
            else 1
        )
    )
)

In [23]:
prf1_patient(fourth_lf_negative, gt_ism_patient)

(0.06593406593406594, 1.0, 0.12371134020618557, 6, 85, 0)

In [24]:
second_fourth = second_lf_unsure.join(
    fourth_lf_negative,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 and i2['payload'] == 1 else -1
    )
)

In [25]:
prf1_patient(
    second_fourth,
    gt_ism_patient
)

(0.1724137931034483, 0.8333333333333334, 0.28571428571428575, 5, 24, 1)

In [26]:
tp, fp, fn = tp_fp_fn_patient(second_fourth, gt_ism_patient)

In [27]:
second_has_zero = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][1]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            -1 if intrvl['payload'].count(0) > 0
            else 1
        )
    )
)

In [28]:
prf1_patient(second_has_zero, gt_ism_patient)

(0.07017543859649122, 0.6666666666666666, 0.12698412698412698, 4, 53, 2)

In [29]:
attempt3 = second_fourth.join(
    second_has_zero,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 and i2['payload'] == 1 else -1
    )
)

In [30]:
prf1_patient(attempt3, gt_ism_patient)

(0.18181818181818182, 0.6666666666666666, 0.28571428571428575, 4, 18, 2)

In [31]:
fifth_lf_starts_or_ends_with_0 = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][4]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            -1 if intrvl['payload'][0] == 0 or intrvl['payload'][-1] == 0
            else 1
        )
    )
)

In [32]:
prf1_patient(fifth_lf_starts_or_ends_with_0, gt_ism_patient)

(0.04597701149425287, 0.6666666666666666, 0.08602150537634408, 4, 83, 2)

In [33]:
attempt4 = attempt3.join(
    fifth_lf_starts_or_ends_with_0,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 and i2['payload'] == 1 else -1
    )
)

In [34]:
prf1_patient(attempt4, gt_ism_patient)

(0.14285714285714285, 0.3333333333333333, 0.2, 2, 12, 4)

In [35]:
tp, fp, fn = tp_fp_fn_patient(attempt4, gt_ism_patient)

In [36]:
three_negatives_twice = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [-1 if intrvl['payload'].count(-1) >= 3 else 1]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            -1 if intrvl['payload'].count(-1) >= 2
            else 1
        )
    )
)

In [37]:
prf1_patient(three_negatives_twice, gt_ism_patient)

(0.0625, 1.0, 0.11764705882352941, 6, 90, 0)

In [38]:
fifth_lf_less_than_three_positive = lf_ism.map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = [intrvl['payload'][4]]
    )
).coalesce(
    ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
).map(
    lambda intrvl: Interval(
        intrvl['bounds'],
        payload = (
            -1 if (intrvl['payload'].count(-1) > 3 or
                  intrvl['payload'].count(-1) > 2 and
                  intrvl['payload'].count(0) == 1)
            else 1
        )
    )
)

In [39]:
prf1_patient(fifth_lf_less_than_three_positive, gt_ism_patient)

(0.10714285714285714, 1.0, 0.19354838709677416, 6, 50, 0)

In [40]:
attempt5 = attempt4.join(
    three_negatives_twice,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 and i2['payload'] == 1 else -1
    )
).join(
    fifth_lf_less_than_three_positive,
    predicate = equal(),
    merge_op = lambda i1, i2: Interval(
        i1['bounds'],
        1 if i1['payload'] == 1 and i2['payload'] == 1 else -1
    )
)

In [41]:
prf1_patient(attempt5, gt_ism_patient)

(0.2, 0.3333333333333333, 0.25, 2, 8, 4)

In [42]:
tp, fp, fn = tp_fp_fn_patient(attempt5, gt_ism_patient)

In [171]:
for k in fp:
    print_lfs(lf_ism[k])

[[ 0. -1.  0.  0. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]]
[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]]


In [172]:
for k in tp:
    print_lfs(lf_ism[k])

[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0. -1.  1.]
 [-1.  1.  0.  0.  1.]
 [-1.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1. -1.  1.]
 [ 1.  1.  0. -1.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]]
[[ 1.  1.  1.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0. -1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]


In [142]:
for k in fp:
    print_lfs(lf_ism[k])

[[-1. -1.  0.  0. -1.]
 [-1.  1.  0. -1. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]
[[ 0. -1.  0.  0. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]]
[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  1.  0. -1.]
 [ 1.  1.  1.  0. -1.]]
[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0.  1.  0.  1.  0.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]


In [143]:
for k in tp:
    print_lfs(lf_ism[k])

[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0. -1.  1.]
 [-1.  1.  0.  0.  1.]
 [-1.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1. -1.  1.]
 [ 1.  1.  0. -1.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]]
[[ 1.  1.  1.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0. -1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]


In [131]:
tp, fp, fn = tp_fp_fn_patient(attempt3, gt_ism_patient)

In [132]:
for k in fp:
    print_lfs(lf_ism[k])

[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1. -1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0.  1.  0.  1.  0.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1. -1.  0.  0. -1.]
 [-1.  1.  0. -1. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]
[[ 0. -1.  0.  0. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]]
[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  1.  0. -1.]
 [ 1.  1.  1.  0. -1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0

In [134]:
for k in tp:
    print_lfs(lf_ism[k])

[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0. -1.  1.]
 [-1.  1.  0.  0.  1.]
 [-1.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1. -1.  1.]
 [ 1.  1.  0. -1.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]]
[[ 0. -1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]]
[[ 1.  1.  1.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0. -1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]


In [119]:
for k in fp:
    print_lfs(lf_ism[k])

[[ 1.  1.  1. -1. -1.]
 [ 0.  0.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]
[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  0.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1. -1.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0. -1.  1.]
 [-1.  1.  0. -1.  1.]
 [ 0.  0.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0.  1.  0.  1.  0.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1. -1.  0.  0. -1.]
 [-1

In [120]:
tp, fp, fn = tp_fp_fn_patient(second_lf_unsure, gt_ism_patient)

In [108]:
sorted(list(fp.get_grouped_intervals().keys()))

[0, 1, 2, 18, 28, 29, 35, 39, 40, 45, 46, 52, 54, 56, 61, 76, 77, 80, 89]

In [110]:
sorted(list(fourth_lf_negative.filter(
    payload_satisfies(lambda p: p == 1)
).get_grouped_intervals().keys()))

[0,
 1,
 2,
 4,
 6,
 7,
 12,
 13,
 18,
 20,
 27,
 28,
 29,
 31,
 32,
 35,
 36,
 39,
 40,
 41,
 42,
 43,
 45,
 46,
 49,
 50,
 52,
 53,
 54,
 56,
 61,
 69,
 70,
 73,
 74,
 75,
 76,
 77,
 80,
 89]

In [98]:
for k in fp:
    print_lfs(lf_ism[k])

[[-1.  1.  0.  0.  1.]
 [-1.  0.  0.  0. -1.]
 [-1.  1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]]
[[ 0.  0.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 1.  1.  1.  0. -1.]
 [ 1.  1.  1.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 1.  1.  1. -1.  1.]]
[[ 1.  1.  1. -1. -1.]
 [ 0.  0.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 1.  1.  0.  0.  1.]]
[[ 0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0. -1.  1.]]
[[-1. -1.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  0.  0.  0. -1.]
 [ 0.  1.  0.  0.  1.]]
[[-1.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[-1. -1.  0.  0.  0.]
 [ 0

In [78]:
gt_ism_patient.filter(payload_satisfies(lambda p: p == 1))

{4: [<Interval t1:0 t2:6 payload:1.0>], 7: [<Interval t1:0 t2:6 payload:1.0>], 22: [<Interval t1:0 t2:6 payload:1.0>], 36: [<Interval t1:0 t2:6 payload:1.0>], 43: [<Interval t1:0 t2:6 payload:1.0>], 53: [<Interval t1:0 t2:6 payload:1.0>], 69: [<Interval t1:0 t2:6 payload:1.0>]}

In [44]:
def print_lfs(iset):
    print(np.array(
        iset.map(
            lambda intrvl: Interval(intrvl['bounds'], [intrvl['payload']])
        ).coalesce(
            ('t1', 't2'), Bounds1D.span, lambda p1, p2: p1 + p2
        ).get_intervals()[0]['payload']))

In [74]:
for k in gt_ism_patient.filter(payload_satisfies(lambda p: p == 1)):
    print_lfs(lf_ism[k])

[[1. 1. 1. 0. 0.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]
 [1. 1. 1. 0. 1.]]
[[ 0.  0.  0.  1.  1.]
 [ 1.  1.  0.  1.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  1.]
 [ 1.  1.  1.  0.  0.]
 [ 1. -1.  1.  0. -1.]]
[[-1.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]]
[[ 0.  1.  0.  1.  0.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  1.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  1. -1.]]
[[1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
[[0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1.]]


In [75]:
for k in gt_ism_patient.filter(payload_satisfies(lambda p: p == -1)):
    print_lfs(lf_ism[k])

[[ 0.  0.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  1.  0. -1.]
 [ 1.  1.  1.  0. -1.]
 [ 1.  1.  1.  0. -1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0. -1.  0.  1.  0.]
 [ 0.  0.  0.  1. -1.]
 [ 0.  0.  0.  1.  1.]
 [ 0. -1.  0.  0.  1.]
 [ 1. -1.  0.  0.  1.]]
[[ 0.  0.  0.  1. -1.]
 [ 0.  1.  0.  1. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0. -1.  0.  0. -1.]
 [ 0. -1.  0.  0. -1.]
 [ 0.  1.  1.  0. -1.]]
[[ 0. -1.  0.  1. -1.]
 [ 0. -1.  0.  0. -1.]
 [ 0. -1.  0.  0. -1.]
 [ 0.  0.  0.  0.  1.]
 [ 0. -1.  1.  0. -1.]
 [ 1. -1.  1.  0. -1.]]
[[ 0. -1.  0.  0. -1.]
 [ 0. -1.  0.  0. -1.]
 [ 0. -1.  0.  1. -1.]
 [ 0. -1.  1.  1. -1.]
 [ 0. -1.  1.  0. -1.]
 [ 1. -1.  1.  0. -1.]]
[[-1. -1.  0.  0. -1.]
 [-1.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]]
[[-1. -1.  0.  0. -1.]
 [ 0. -1.  0.  0. -1.]
 [ 0. -1.  0. -1.  0.]
 [ 0.  1.  0. -1. -1.]
 [ 0.  1.  0.  0.  1.]
 [ 0.  1.  0. -1.  1.]]
[[ 0. -1.  0.  0. -1.]
 [ 0